In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import warnings
warnings.filterwarnings('ignore')
import emojis
import unicodedata
from bs4 import BeautifulSoup
from langdetect import detect, DetectorFactory
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


/Users/tienhuynh/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [2]:
address = pd.read_excel("./address.xlsx")
address

,id,addressObj/postalcode
0,239853,77019
1,1158926,77079
2,111618,77046
3,223137,77092-7307
4,223151,77054
...,...,...
1483,653244,77056-6301
1484,1642905,77003
1485,99014,77092-7312
1486,99104,77060-3507


In [3]:
review = pd.read_csv("./reviews.csv")
review

,id_x,name_x,ai_review,no of amenities,description,hotelClass,nearestMetroStations/0/distance,nearestMetroStations/1/distance,nearestMetroStations/2/distance,nearestMetroStations/3/distance,...,via_mobile,service,cleanliness,overall,value,location,sleep_quality,rooms,check_in_front_desk,business_service_(e_g_internet_access)
0,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,NaN
2,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
3,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,5.0,4.0,NaN,NaN
4,113317,Casablanca Hotel Times Square,"Nestled conveniently near Times Square, Casabl...",50,Experience the magic of Times Square just step...,4.0,115.85,133.62,191.48,209.85,...,False,5.0,5.0,5.0,5.0,5.0,4.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274465,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN
274466,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,4.0,4.0,4.0,4.0,4.0,NaN,4.0,4.0,3.0
274467,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,4.0,5.0,4.0,NaN,NaN,NaN,5.0,NaN,NaN
274468,235513,Sofitel Washington DC,Sofitel Washington DC Lafayette Square has rec...,79,Sofitel Washington DC Lafayette Square represe...,4.0,187.75,461.26,510.73,572.33,...,False,5.0,5.0,5.0,5.0,5.0,NaN,5.0,5.0,NaN


In [4]:
review = pd.merge(left=review, right=address, left_on="id_x", right_on="id")

In [5]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274470 entries, 0 to 274469
Data columns (total 45 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   id_x                                    274470 non-null  int64  
 1   name_x                                  274470 non-null  object 
 2   ai_review                               190401 non-null  object 
 3   no of amenities                         274470 non-null  int64  
 4   description                             261606 non-null  object 
 5   hotelClass                              274470 non-null  float64
 6   nearestMetroStations/0/distance         274470 non-null  float64
 7   nearestMetroStations/1/distance         274470 non-null  float64
 8   nearestMetroStations/2/distance         274470 non-null  float64
 9   nearestMetroStations/3/distance         274470 non-null  float64
 10  nearestMetroStations/4/distance         2744

In [6]:
metro_columns = review.columns[5:10]
review['dist_to_metro'] = review[metro_columns].mean(axis=1)

In [7]:
review['no_of_attractions'] = review['neighborhood location'].apply(
    lambda x: len(x.split(',')) if isinstance(x, str) else 0)

In [8]:
review.drop(columns=["ai_review", 
                     "photoCount", 
                     "priceRange", 
                     "rankingDenominator", 
                     "rankingPosition", 
                     "rating",
                     "roomTips/0/rating", 
                     "roomTips/0/text", 
                     "roomTips/1/rating", 
                     'roomTips/1/text', 
                     "roomTips/2/rating", 
                     "roomTips/2/text", "roomTips/3/rating", 
                     "roomTips/3/text", "roomTips/4/rating", 
                     "roomTips/4/text", "title","nearestMetroStations/0/distance", 
                     "nearestMetroStations/1/distance", "nearestMetroStations/2/distance", 
                     "nearestMetroStations/3/distance", "nearestMetroStations/4/distance", 
                     "neighborhood location", "numberOfRooms", "via_mobile"
                     ,"id"
                    ], inplace=True)

In [9]:
# Define the mapping
price_mapping = {
    '$': "low",         # Low price
    '$$': "moderate",        # Moderate price
    '$$$': "expensive",       # Expensive
    '$$$$': "very expensive"       # Very expensive
}

# Apply the mapping to the 'priceLevel' column
review['priceLevel'] = review['priceLevel'].map(price_mapping)

In [10]:
# Convert 'award' to 1/0 based on whether a value exists
review['travelerChoiceAward'] = np.where(review['travelerChoiceAward'].notna() & (review['travelerChoiceAward'] != ''), 1, 0)


In [11]:
# Function to convert CamelCase or PascalCase to snake_case
def camel_to_snake(name):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

# Apply the function to all column names
review.columns = [camel_to_snake(col) for col in review.columns]

In [12]:
review = review.rename(columns={"id_x": "id", 
                                "name_x":"name", 
                                "no of amenities": "no_of_amenities", 
                                "number_of_reviews": "no_of_reviews",
                                "traveler_choice_award": "award",
                                "text": "review",
                                "business_service_(e_g_internet_access)": "business_service",
                                "overall": "ratings",
                                "address_obj/postalcode": "postal_code"
                                
                               })

In [13]:
new_order = ["id",
             "name",
             "postal_code",
             "description",
             "review",
             "hotel_class",
             "no_of_amenities",
             "price_level",
             "dist_to_metro",
             "no_of_attractions",
             "award",
             "no_of_reviews",
             "service",
             "cleanliness",
             "value",
             "location",
             "sleep_quality",
             "rooms",
             "check_in_front_desk",
             "business_service",
             "ratings"
            ]
review = review[new_order]

In [14]:
# Calculate the percentage of null values in each column
null_percentage = pd.DataFrame({
    'column': review.columns,
    'null_percentage': round((review.isnull().mean() * 100),2)
}).reset_index(drop=True)
null_percentage

,column,null_percentage
0,id,0.00
1,name,0.00
2,postal_code,0.01
3,description,4.69
4,review,0.00
5,hotel_class,0.00
6,no_of_amenities,0.00
7,price_level,1.23
8,dist_to_metro,0.00
9,no_of_attractions,0.00


In [15]:
# Drop rows where 'postal_code' is NaN
review.dropna(subset=['postal_code'], inplace=True)

In [16]:
review[review['description'].isna()]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings
12634,1218737,La Quinta Inn & Suites Houston - Normandy,77015-4920,NaN,"This is a very nice place to stay, very clean ...",3.0,32,low,0.600,0,...,185,5.0,5.0,5.0,3.0,5.0,5.0,NaN,NaN,5.0
12635,1218737,La Quinta Inn & Suites Houston - Normandy,77015-4920,NaN,Excellent and caring front desk staff and mana...,3.0,32,low,0.600,0,...,185,5.0,5.0,4.0,4.0,5.0,5.0,NaN,NaN,5.0
12636,1218737,La Quinta Inn & Suites Houston - Normandy,77015-4920,NaN,This is a very clean Hotel and the service was...,3.0,32,low,0.600,0,...,185,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,5.0
12637,1218737,La Quinta Inn & Suites Houston - Normandy,77015-4920,NaN,Location was of the most up most importance Ce...,3.0,32,low,0.600,0,...,185,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,4.0
12638,1218737,La Quinta Inn & Suites Houston - Normandy,77015-4920,NaN,"People if you're in Houston,Texas and need a h...",3.0,32,low,0.600,0,...,185,5.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260683,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Convenient if you don't mind walking a few blo...,4.0,96,expensive,567.482,3,...,1369,4.0,4.0,3.0,NaN,NaN,4.0,NaN,NaN,4.0
260684,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Very attentive and accommodating staff. Initia...,4.0,96,expensive,567.482,3,...,1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
260685,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Ottimo hotel. Il personale è gentilissimo. La ...,4.0,96,expensive,567.482,3,...,1369,3.0,4.0,3.0,4.0,3.0,3.0,NaN,NaN,5.0
260686,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Non centralissimo ma molto vicino alla metro. ...,4.0,96,expensive,567.482,3,...,1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [17]:
review[review['name'] == "Renaissance Washington, DC Dupont Circle Hotel"]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings
260337,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Stayed here several times this year for busine...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,5.0,5.0,3.0,NaN,NaN,4.0
260338,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Just spent the weekend at this hotel. The Dupo...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,5.0,4.0,3.0,NaN,NaN,4.0
260339,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,A few blocks' walk from Dupont Circle Metro st...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,4.0
260340,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,"This hotel is nicely updated with very chic, m...",4.0,96,expensive,567.482,3,...,1369,4.0,4.0,5.0,5.0,4.0,4.0,NaN,NaN,4.0
260341,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,The global hotel cuality is good but there are...,4.0,96,expensive,567.482,3,...,1369,5.0,5.0,4.0,5.0,5.0,4.0,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260683,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Convenient if you don't mind walking a few blo...,4.0,96,expensive,567.482,3,...,1369,4.0,4.0,3.0,NaN,NaN,4.0,NaN,NaN,4.0
260684,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Very attentive and accommodating staff. Initia...,4.0,96,expensive,567.482,3,...,1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
260685,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Ottimo hotel. Il personale è gentilissimo. La ...,4.0,96,expensive,567.482,3,...,1369,3.0,4.0,3.0,4.0,3.0,3.0,NaN,NaN,5.0
260686,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,NaN,Non centralissimo ma molto vicino alla metro. ...,4.0,96,expensive,567.482,3,...,1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [18]:
# Fill missing descriptions with the mode description for the same hotel class and postal code
review['description'] = review.groupby(['hotel_class', 'postal_code'])['description'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "No description available")
)

In [19]:
review['price_level'].fillna("Not Specified", inplace=True)

In [20]:
# Define columns and their respective imputation methods
columns_to_impute = {
    'cleanliness': 'median',
    'service': 'median',
    'location': 'median',
    'value': 'mean',
    'sleep_quality': 'median',
    'rooms': 'mean',
    'check_in_front_desk': 'median',
    'business_service': 'median'
}

# Impute null values using apply
for column, method in columns_to_impute.items():
    review[column] = review[column].fillna(getattr(review[column], method)())
    review[column] = review[column].round(1)

In [21]:
# Calculate the percentage of null values in each column
null_percentage = pd.DataFrame({
    'column': review.columns,
    'null_percentage': round((review.isnull().mean() * 100),2)
}).reset_index(drop=True)
null_percentage

,column,null_percentage
0,id,0.0
1,name,0.0
2,postal_code,0.0
3,description,0.0
4,review,0.0
5,hotel_class,0.0
6,no_of_amenities,0.0
7,price_level,0.0
8,dist_to_metro,0.0
9,no_of_attractions,0.0


In [22]:
review[review['name'] == "Renaissance Washington, DC Dupont Circle Hotel"]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings
260337,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Stayed here several times this year for busine...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,5.0,5.0,3.0,4.0,4.0,4.0
260338,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Just spent the weekend at this hotel. The Dupo...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,5.0,4.0,3.0,4.0,4.0,4.0
260339,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,A few blocks' walk from Dupont Circle Metro st...,4.0,96,expensive,567.482,3,...,1369,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
260340,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,"This hotel is nicely updated with very chic, m...",4.0,96,expensive,567.482,3,...,1369,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0
260341,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,The global hotel cuality is good but there are...,4.0,96,expensive,567.482,3,...,1369,5.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260683,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Convenient if you don't mind walking a few blo...,4.0,96,expensive,567.482,3,...,1369,4.0,4.0,3.0,5.0,5.0,4.0,4.0,4.0,4.0
260684,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Very attentive and accommodating staff. Initia...,4.0,96,expensive,567.482,3,...,1369,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,4.0
260685,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Ottimo hotel. Il personale è gentilissimo. La ...,4.0,96,expensive,567.482,3,...,1369,3.0,4.0,3.0,4.0,3.0,3.0,4.0,4.0,5.0
260686,84101,"Renaissance Washington, DC Dupont Circle Hotel",20037,Welcome to a city that is perpetually changing...,Non centralissimo ma molto vicino alla metro. ...,4.0,96,expensive,567.482,3,...,1369,5.0,5.0,3.9,5.0,5.0,4.0,4.0,4.0,4.0


In [23]:
review = review.drop_duplicates(keep="first", subset=['review'])

In [24]:
review 

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,no_of_reviews,service,cleanliness,value,location,sleep_quality,rooms,check_in_front_desk,business_service,ratings
0,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,We had a whisper-quiet queen deluxe room couty...,4.0,50,very expensive,130.960,5,...,7596,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0
1,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,Great 3 night stay right after Thanksgiving. A...,4.0,50,very expensive,130.960,5,...,7596,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0
2,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,I hesitate to give this little gem a stellar r...,4.0,50,very expensive,130.960,5,...,7596,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0
3,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,You won't find a more conveniently located hot...,4.0,50,very expensive,130.960,5,...,7596,5.0,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0
4,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,This small boutique hotel right off Times Squa...,4.0,50,very expensive,130.960,5,...,7596,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274465,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Viel mehr kann man eigentlich auch gar nicht s...,4.0,79,very expensive,347.214,3,...,3973,5.0,5.0,3.9,5.0,5.0,5.0,4.0,4.0,5.0
274466,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Wir können nur Gutes schreiben: Sehr netter Em...,4.0,79,very expensive,347.214,3,...,3973,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,4.0
274467,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Das Sofitel Lafayette Square liegt in unmittel...,4.0,79,very expensive,347.214,3,...,3973,4.0,5.0,3.9,5.0,5.0,5.0,4.0,4.0,4.0
274468,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Nous avons sejourne 2 nuits au Sofitel de Wash...,4.0,79,very expensive,347.214,3,...,3973,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0


In [25]:
def normalize_text(text: str, lower: bool = True) -> str:
    """
    Normalize text by lowercasing it and using the NFD method for representation.

    Args:
        text (str): comment
        lower (bool): whether to lowercase the text

    Returns:
        str: normalized comment
    """
    text = text.lower() if lower else text
    return unicodedata.normalize('NFD', text)


In [26]:
# List of columns to apply the normalization function
columns_to_normalize = ['review', 'description', 'name']

# Apply the normalization function and create new columns with the prefix "normalized_"
for column in columns_to_normalize:
    review[f'normalized_{column}'] = review[column].apply(normalize_text)

In [27]:
# Define the URL pattern
URL_PATTERN = unicodedata.normalize(
    'NFD', 
    r"(?i)\b((?:http[s]?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)"
    r"(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
)

# Define the HTML tag pattern
HTML_PATTERN = r'<[^>]+>'

def contains_url_or_html(text: str) -> int:
    """
    Check if the text contains a URL or HTML tag.

    Args:
        text (str): comment

    Returns:
        int: 1 if URL or HTML tag is present, 0 otherwise
    """
    url_flag = re.search(URL_PATTERN, text)
    html_flag = re.search(HTML_PATTERN, text)
    return int(url_flag is not None or html_flag is not None)

In [28]:
columns_contains_url = ['normalized_review', 'normalized_description', 'normalized_name']

# Apply the normalization function 
for column in columns_contains_url:
    review[f'url_{column}'] = review[column].apply(contains_url_or_html)

In [29]:
review[review['url_normalized_review']==1]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,rooms,check_in_front_desk,business_service,ratings,normalized_review,normalized_description,normalized_name,url_normalized_review,url_normalized_description,url_normalized_name
1543,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,I'll start by saying that I prefer small hotel...,4.0,50,very expensive,130.960,5,...,4.0,5.0,5.0,5.0,i'll start by saying that i prefer small hotel...,experience the magic of times square just step...,casablanca hotel times square,1,0,0
2116,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,I stayed at the Hotel Casablanca for a busines...,4.0,50,very expensive,130.960,5,...,4.0,4.0,4.0,4.0,i stayed at the hotel casablanca for a busines...,experience the magic of times square just step...,casablanca hotel times square,1,0,0
2490,217616,The Muse Hotel New York,10036-8501,"Discover an enduring haven for connection, ins...",We stayed at the Muse in June during a very ho...,4.0,63,very expensive,235.750,4,...,4.0,4.0,4.0,4.0,we stayed at the muse in june during a very ho...,"discover an enduring haven for connection, ins...",the muse hotel new york,1,0,0
4001,208454,Sofitel New York,10036,**Gaby Brasserie is open for Breakfast from 6:...,Cannot recommend this hotel highly enough. We ...,4.0,88,very expensive,227.348,3,...,5.0,4.0,4.0,5.0,cannot recommend this hotel highly enough. we ...,**gaby brasserie is open for breakfast from 6:...,sofitel new york,1,0,0
4421,208454,Sofitel New York,10036,**Gaby Brasserie is open for Breakfast from 6:...,I stayed in this hotel for 5 nights at the end...,4.0,88,very expensive,227.348,3,...,5.0,4.0,4.0,5.0,i stayed in this hotel for 5 nights at the end...,**gaby brasserie is open for breakfast from 6:...,sofitel new york,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272056,84093,"Palomar Washington DC, a Kimpton Hotel",20037,The Royal Sonesta Washington DC Dupont Circle ...,"After a lot of research, we decided on the Pal...",4.5,69,expensive,711.572,2,...,5.0,4.0,4.0,5.0,"after a lot of research, we decided on the pal...",the royal sonesta washington dc dupont circle ...,"palomar washington dc, a kimpton hotel",1,0,0
272521,84093,"Palomar Washington DC, a Kimpton Hotel",20037,The Royal Sonesta Washington DC Dupont Circle ...,Great location right in down the street from D...,4.5,69,expensive,711.572,2,...,5.0,4.0,4.0,5.0,great location right in down the street from d...,the royal sonesta washington dc dupont circle ...,"palomar washington dc, a kimpton hotel",1,0,0
272848,84093,"Palomar Washington DC, a Kimpton Hotel",20037,The Royal Sonesta Washington DC Dupont Circle ...,"Good hotel. \nRoom was well furnished, clean a...",4.5,69,expensive,711.572,2,...,4.0,4.0,4.0,5.0,"good hotel. \nroom was well furnished, clean a...",the royal sonesta washington dc dupont circle ...,"palomar washington dc, a kimpton hotel",1,0,0
272914,84093,"Palomar Washington DC, a Kimpton Hotel",20037,The Royal Sonesta Washington DC Dupont Circle ...,We just returned from another 4 day/3 night st...,4.5,69,expensive,711.572,2,...,4.0,4.0,4.0,4.0,we just returned from another 4 day/3 night st...,the royal sonesta washington dc dupont circle ...,"palomar washington dc, a kimpton hotel",1,0,0


In [30]:
def remove_html_tags_and_urls(text):
    # Remove HTML tags
    text_without_html = re.sub(r'<[^>]+>', '', text)
    # Remove URLs starting with http:// or https://
    text_cleaned = re.sub(r'http[s]?://\S+|www\.\S+', '', text_without_html)
    return text_cleaned


In [31]:
# Check if 'normalized_review' exists and apply the function
if 'normalized_review' in review.columns:
    review['normalized_review'] = review['normalized_review'].apply(remove_html_tags_and_urls)
else:
    print("The 'normalized_review' column does not exist in the DataFrame.")

In [32]:
# Pattern for English uppercase letters


def containAdvertisement(text: str) -> int:
    """
    Check if uppercase letters occupy more than half the length of the string.

    Args:
        ptext (str): review text

    Returns:
        int: 1 if uppercase letters occupy more than half the length of the string, otherwise 0
    """
    upper_pattern = r'[A-Z]'
    # Count the number of uppercase letters in the text
    upper_count = len(re.findall(upper_pattern, text))
    
    # Check if uppercase letters occupy more than half the text length
    if len(text) > 0 and upper_count > 0.5 * len(text):
        return 1
    else:
        return 0

In [33]:
review['contain_adv'] = review['normalized_review'].apply(containAdvertisement)
review['contain_adv_1'] = review['normalized_description'].apply(containAdvertisement)

In [34]:
review[(review['contain_adv'] == 1) | (review['contain_adv_1'] == 1)]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,business_service,ratings,normalized_review,normalized_description,normalized_name,url_normalized_review,url_normalized_description,url_normalized_name,contain_adv,contain_adv_1


In [35]:
def extract_and_remove_emoji(text: str) -> (str, str):
    """
    Extract emojis from the comment and return the cleaned text without emojis.

    Args:
        text (str): comment

    Returns:
        tuple: (string containing extracted emojis, string without emojis)
    """
    # Regex pattern to match emojis
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F7FF"  # alchemical symbols
        u"\U0001F800-\U0001F8FF"  # geometric shapes
        u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        u"\U0001FA00-\U0001FAFF"  # chess symbols
        u"\U00002700-\U000027BF"  # dingbats
        "]+", 
        flags=re.UNICODE
    )
    
    # Find all emojis in the text
    extracted_emojis = emoji_pattern.findall(text)
    # Join the extracted emojis into a single string
    emoji_string = ' '.join(extracted_emojis)
    
    # Remove emojis from the text
    cleaned_text = emoji_pattern.sub('', text)
    
    return cleaned_text


In [36]:
review['normalized_review'] = review['normalized_review'].apply(extract_and_remove_emoji)

In [37]:
def extractEmoji(text: str) -> str:
    """
    Extract emojis from the comment.

    Args:
        text (str): comment

    Returns:
        str: a string containing the extracted emojis
    """
    # Regex pattern to match emojis
    emoji_pattern = re.compile(
        "[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F7FF"  # alchemical symbols
        u"\U0001F800-\U0001F8FF"  # geometric shapes
        u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        u"\U0001FA00-\U0001FAFF"  # chess symbols
        u"\U00002700-\U000027BF"  # dingbats
        "]+", 
        flags=re.UNICODE
    )
    
    # Find all emojis in the text
    extracted_emojis = emoji_pattern.findall(text)
    return ' '.join(extracted_emojis)

In [38]:
review['emo'] = review['normalized_review'].apply(extractEmoji)

In [39]:
review['emo'].value_counts()

emo
    274147
Name: count, dtype: int64

In [40]:
# Set seed for reproducibility
DetectorFactory.seed = 0

def is_english(text: str) -> bool:
    """
    Check if the given text is in English.

    Args:
        text (str): The text to check.

    Returns:
        bool: True if the text is in English, False otherwise.
    """
    try:
        return detect(text) == 'en'
    except:
        return False  # Return False if detection fails


In [41]:
# review['isEnglish'] = review['normalized_review'].apply(is_english)

In [42]:
# non_english_indices = review[review['isEnglish'] == False].index.tolist()

# # Step 2: Optionally save the indices list to a file (CSV, JSON, etc.)
# # For example, saving to a CSV:

# non_english_df = pd.DataFrame(non_english_indices, columns=['index'])
# non_english_df.to_csv('non_english_indices.csv', index=False) 

In [43]:
non_english = pd.read_csv("./non_english_indices.csv")

In [44]:
non_english

,index
0,587
1,2195
2,2196
3,2197
4,2198
...,...
34976,274465
34977,274466
34978,274467
34979,274468


In [45]:
review.loc[non_english['index']]

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,ratings,normalized_review,normalized_description,normalized_name,url_normalized_review,url_normalized_description,url_normalized_name,contain_adv,contain_adv_1,emo
587,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,It was amazing experience of NY by staying at ...,4.0,50,very expensive,130.960,5,...,5.0,it was amazing experience of ny by staying at ...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
2195,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,"Tout est parfait dans ce restaurant, le petit ...",4.0,50,very expensive,130.960,5,...,4.0,"tout est parfait dans ce restaurant, le petit ...",experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
2196,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,Suveränt trevlig och service inriktad personal...,4.0,50,very expensive,130.960,5,...,5.0,suveränt trevlig och service inriktad persona...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
2197,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,"Nos alojamos 1 semana, mi esposo, mi niño de 4...",4.0,50,very expensive,130.960,5,...,5.0,"nos alojamos 1 semana, mi esposo, mi niño de ...",experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
2198,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,Ett mer välkomnande och servicevänligt hotell ...,4.0,50,very expensive,130.960,5,...,5.0,ett mer välkomnande och servicevänligt hotel...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274465,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Viel mehr kann man eigentlich auch gar nicht s...,4.0,79,very expensive,347.214,3,...,5.0,viel mehr kann man eigentlich auch gar nicht s...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
274466,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Wir können nur Gutes schreiben: Sehr netter Em...,4.0,79,very expensive,347.214,3,...,4.0,wir können nur gutes schreiben: sehr netter e...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
274467,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Das Sofitel Lafayette Square liegt in unmittel...,4.0,79,very expensive,347.214,3,...,4.0,das sofitel lafayette square liegt in unmittel...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
274468,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Nous avons sejourne 2 nuits au Sofitel de Wash...,4.0,79,very expensive,347.214,3,...,5.0,nous avons sejourne 2 nuits au sofitel de wash...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,


In [46]:
review = review.drop(non_english['index']).reset_index(drop=True)

In [47]:
review

,id,name,postal_code,description,review,hotel_class,no_of_amenities,price_level,dist_to_metro,no_of_attractions,...,ratings,normalized_review,normalized_description,normalized_name,url_normalized_review,url_normalized_description,url_normalized_name,contain_adv,contain_adv_1,emo
0,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,We had a whisper-quiet queen deluxe room couty...,4.0,50,very expensive,130.960,5,...,5.0,we had a whisper-quiet queen deluxe room couty...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
1,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,Great 3 night stay right after Thanksgiving. A...,4.0,50,very expensive,130.960,5,...,5.0,great 3 night stay right after thanksgiving. a...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
2,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,I hesitate to give this little gem a stellar r...,4.0,50,very expensive,130.960,5,...,5.0,i hesitate to give this little gem a stellar r...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
3,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,You won't find a more conveniently located hot...,4.0,50,very expensive,130.960,5,...,5.0,you won't find a more conveniently located hot...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
4,113317,Casablanca Hotel Times Square,10036-6575,Experience the magic of Times Square just step...,This small boutique hotel right off Times Squa...,4.0,50,very expensive,130.960,5,...,5.0,this small boutique hotel right off times squa...,experience the magic of times square just step...,casablanca hotel times square,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239161,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Beautifully decorated and comfortable rooms ev...,4.0,79,very expensive,347.214,3,...,5.0,beautifully decorated and comfortable rooms ev...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
239162,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,I was there with my girlfriend for 4 days at t...,4.0,79,very expensive,347.214,3,...,5.0,i was there with my girlfriend for 4 days at t...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
239163,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,Upscale hotel in downtown DC. Walking distance...,4.0,79,very expensive,347.214,3,...,4.0,upscale hotel in downtown dc. walking distance...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,
239164,235513,Sofitel Washington DC,20005,Sofitel Washington DC Lafayette Square represe...,All expectations for service and accommodation...,4.0,79,very expensive,347.214,3,...,5.0,all expectations for service and accommodation...,sofitel washington dc lafayette square represe...,sofitel washington dc,0,0,0,0,0,


In [48]:
def process_text(text):
    # Remove all special characters
    processed_feature = re.sub(r'\W', ' ', str(text))
    
    # Remove all words that include digits / numbers
    processed_feature = re.sub(r'\w*\d\w*', ' ', processed_feature)
    
    # Remove all single characters
    processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    
    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    
    # Substituting multiple spaces with a single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)
    
    # Return the cleaned text
    return processed_feature.strip()

# Step 2: Apply the function to 3 columns in the DataFrame
columns_to_process = ['normalized_review', 'normalized_description', 'normalized_name']  # Replace with your column names
review[columns_to_process] = review[columns_to_process].applymap(process_text)


In [49]:
review.drop(columns=["url_normalized_review", 
                     "url_normalized_description", 
                     "url_normalized_name",
                     "contain_adv", 
                     "contain_adv_1", 
                     "emo",
                     "postal_code"
                    ], inplace=True)

In [50]:
# Get the set of English stopwords
stops = set(stopwords.words('english'))

# Define a function to remove stopwords
def remove_stopwords(text):
    words = word_tokenize(text)  # Tokenize the text
    filtered_words = [word for word in words if word.lower() not in stops]  # Remove stopwords
    return ' '.join(filtered_words)  # Join the words back into a string

# Apply the function to the specified columns
columns_to_clean = ['normalized_review', 'normalized_description', 'normalized_name']
review[columns_to_clean] = review[columns_to_clean].applymap(remove_stopwords)


In [51]:
# Get the stem text
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def stem_text(text):
    
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(text)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)

In [52]:
# Apply the function to the specified columns
columns_to_stem = ['normalized_review', 'normalized_description', 'normalized_name']
review[columns_to_stem] = review[columns_to_stem].applymap(stem_text)

In [53]:
review.to_csv("review_cleaned.csv",index=False)

In [54]:
review['ratings'].value_counts()

ratings
5.0    101589
4.0     76777
3.0     31063
2.0     15680
1.0     14057
Name: count, dtype: int64

In [55]:
def categorize_rating(rating):
    if rating <= 2.0:
        return 0
    elif rating ==3 :
        return 2
    else:
        return 1

# Apply the categorization function to the ratings column
review['target'] = review['ratings'].apply(categorize_rating)

In [56]:
review['target'].value_counts()

target
1    178366
2     31063
0     29737
Name: count, dtype: int64

In [57]:
X = review['normalized_review']
y = review['target']

In [58]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [59]:
y_train.value_counts(normalize=True)

target
1    0.745753
2    0.130800
0    0.123447
Name: proportion, dtype: float64

In [60]:
#Downsampling to avoid overfitting since there is imbalance among classes

# Combine X_train and y_train into a single DataFrame for resampling
train = pd.concat([X_train, y_train], axis=1)

# Separate the classes
majority_class = train[train['target'] == 1]  
minority_class_1 = train[train['target'] == 0]  
minority_class_2 = train[train['target'] == 2]  

# Downsample the majority class
majority_downsampled = resample(majority_class, 
                                replace=False,    # sample without replacement
                                n_samples=len(minority_class_1),  # match to the larger of the minority classes
                                random_state=42)

# Upsample both minority classes to match the downsampled majority class size
minority_class_1_upsampled = resample(minority_class_1, 
                                      replace=True,    # sample with replacement
                                      n_samples=len(minority_class_1),  # adjust size based on needs
                                      random_state=42)

minority_class_2_upsampled = resample(minority_class_2, 
                                      replace=True,    # sample with replacement
                                      n_samples=len(minority_class_1),  # adjust size based on needs
                                      random_state=42)

# Combine the resampled majority and minority classes
train_balanced = pd.concat([majority_downsampled, minority_class_1_upsampled, minority_class_2_upsampled])

# Shuffle the new training data
train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate the features and target again
X_train_balanced = train_balanced.drop('target', axis=1)
y_train_balanced = train_balanced['target']


In [61]:
y_train_balanced.value_counts(normalize=True)

target
0    0.333333
1    0.333333
2    0.333333
Name: proportion, dtype: float64

In [62]:
#Countvectorizer
# Use count vectorizer to check how many unique words there are
cvec = CountVectorizer(stop_words='english') 
cvec_df = pd.DataFrame(cvec.fit_transform(X_train_balanced['normalized_review']).todense(), columns=cvec.get_feature_names_out())
cvec_df.shape

(62001, 44760)

In [63]:
def cmat(actual_y, predictions, dataset):
    # Check the unique classes in the actual values and predictions
    unique_classes = np.unique(np.concatenate((actual_y, predictions)))
    num_classes = len(unique_classes)
    
    # Create a classification report
    print('Classification report for', dataset)
    print(classification_report(actual_y, predictions))
    print('')
    
    # Create a confusion matrix
    cm = confusion_matrix(actual_y, predictions)
    
    # Dynamically create labels for the confusion matrix
    labels = [f'Class {cls}' for cls in unique_classes]
    
    # Convert confusion matrix to DataFrame
    cm_df = pd.DataFrame(cm, columns=[f'Predicted {label}' for label in labels], 
                         index=[f'Actual {label}' for label in labels])
    
    print('Confusion matrix for', dataset)
    print(cm_df)


In [64]:
# Create a pipeline with Count Vectorizer and Logistic Regression
pipe_cvec_lr = Pipeline([
    ('cvec', CountVectorizer(stop_words='english')), 
    ('lr', LogisticRegression(random_state=42))
])

# Search over the following values of hyperparameters:
pipe_cvec_lr_params = {
    'cvec__max_features': [300], #100,200
    'cvec__min_df': [0.001, 0.005], 
    'cvec__max_df': [.7, .9], 
#     'cvec__ngram_range':[(1,1),(1,2)],  
    'lr__penalty': ['l2'],
    'lr__C': [.01,.1]
}

# Instantiate GridSearchCV
gs_cvec_lr = GridSearchCV(pipe_cvec_lr, # Objects to optimise
                          param_grid = pipe_cvec_lr_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_cvec_lr.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
cvec_lr_pred = gs_cvec_lr.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_cvec_lr.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_cvec_lr.best_score_)
print('Training score:', gs_cvec_lr.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_cvec_lr.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, cvec_lr_pred, 'validation set')

Best parameters:  {'cvec__max_df': 0.9, 'cvec__max_features': 300, 'cvec__min_df': 0.001, 'lr__C': 0.1, 'lr__penalty': 'l2'}
Best CV score:  0.7034241953691133
Training score: 0.7096014580409994
Validation score: 0.7567665505226481

Classification report for validation set
              precision    recall  f1-score   support

           0       0.60      0.75      0.67      9070
           1       0.94      0.79      0.86     53515
           2       0.32      0.55      0.41      9165

    accuracy                           0.76     71750
   macro avg       0.62      0.70      0.65     71750
weighted avg       0.82      0.76      0.78     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1  Predicted Class 2
Actual Class 0               6819                456               1795
Actual Class 1               2327              42479               8709
Actual Class 2               2138               2027               5000


In [65]:
# Create a pipeline with Count Vectorizer and Naive Bayes
pipe_cvec_nb = Pipeline([
    ('cvec', CountVectorizer(stop_words='english')),
    ('nb', MultinomialNB())
])

# Search over the following values of hyperparameters:
pipe_cvec_nb_params = {
    'cvec__max_features': [500], #200
    'cvec__min_df': [0.001, 0.005],
    'cvec__max_df': [.9,.95], 
#     'cvec__ngram_range':[(1,1),(1,2)],  
}

# Instantiate GridSearchCV
gs_cvec_nb = GridSearchCV(pipe_cvec_nb, # Objects to optimise
                          param_grid = pipe_cvec_nb_params, # Hyperparameters for tuning
                          cv=10) # 10-fold cross validation

# Fit model on to training data
gs_cvec_nb.fit(X_train_balanced.values.ravel(), y_train_balanced)

# Generate predictions on validation set
cvec_nb_pred = gs_cvec_nb.predict(X_test)

# Print best parameters
print('Best parameters: ', gs_cvec_nb.best_params_)

# Print accuracy scores
print('Best CV score: ', gs_cvec_nb.best_score_)
print('Training score:', gs_cvec_nb.score(X_train_balanced.values.ravel(), y_train_balanced))
print('Validation score:', gs_cvec_nb.score(X_test, y_test))
print('')

# Print classification report and confusion matrix
cmat(y_test, cvec_nb_pred, 'validation set')

Best parameters:  {'cvec__max_df': 0.9, 'cvec__max_features': 500, 'cvec__min_df': 0.001}
Best CV score:  0.6951178972174102
Training score: 0.6970048870179513
Validation score: 0.7602090592334495

Classification report for validation set
              precision    recall  f1-score   support

           0       0.61      0.72      0.66      9070
           1       0.94      0.80      0.87     53515
           2       0.33      0.56      0.41      9165

    accuracy                           0.76     71750
   macro avg       0.63      0.69      0.65     71750
weighted avg       0.82      0.76      0.78     71750


Confusion matrix for validation set
                Predicted Class 0  Predicted Class 1  Predicted Class 2
Actual Class 0               6504                430               2136
Actual Class 1               2255              42891               8369
Actual Class 2               1878               2137               5150


In [66]:
# # Create a pipeline with Count Vectorizer and SVC
# pipe_cvec_svc = Pipeline([
#     ('cvec', CountVectorizer(stop_words='english')),
#     ('svc', SVC(random_state=42))
# ])

# # Search over the following values of hyperparameters:
# pipe_cvec_svc_params = {
#     'cvec__max_features': [300], #200,500
#     'cvec__min_df': [0.001, 0.005], 
#     'cvec__max_df': [.9,.95], 
# #     'cvec__ngram_range':[(1,1),(1,2)],  
#     'svc__kernel': ['linear'], #'poly', 'rbf'
# #     'svc__degree': [3],
#     'svc__C': [.1]
# }

# # Instantiate GridSearchCV
# gs_cvec_svc = GridSearchCV(pipe_cvec_svc, # Objects to optimise
#                           param_grid = pipe_cvec_svc_params, # Hyperparameters for tuning
#                           cv=10) # 10-fold cross validation

# # Fit model on to training data
# gs_cvec_svc.fit(X_train_balanced.values.ravel(), y_train_balanced)

# # Generate predictions on validation set
# cvec_svc_pred = gs_cvec_svc.predict(X_test)

# # Print best parameters
# print('Best parameters: ', gs_cvec_svc.best_params_)

# # Print accuracy scores
# print('Best CV score: ', gs_cvec_svc.best_score_)
# print('Training score:', gs_cvec_svc.score(X_train_balanced.values.ravel(), y_train_balanced))
# print('Validation score:', gs_cvec_svc.score(X_test, y_test))
# print('')

# # Print classification report and confusion matrix
# cmat(y_test, cvec_svc_pred, 'validation set')